In [ ]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.mllib.recommendation import ALS,MatrixFactorizationModel,Rating
from math import sqrt
import datetime

In [ ]:
print datetime.datetime.now()

In [ ]:
sc = SparkContext()

In [ ]:
data = sc.textFile('/home/ubuntu/client3_cust_prod')
print data.take(5)

In [ ]:
data = data.zipWithIndex().filter(lambda tup: tup[1] > 0).map(lambda tup: tup[0])

In [ ]:
print data.first()
print data.count()

In [ ]:
clean_data = data.map(lambda x:x.split(','))
print clean_data.take(5)

In [ ]:
rating = clean_data.map(lambda y: int(y[3]))
print rating.take(5)

In [ ]:
cust_id = clean_data.map(lambda y : int(y[2]))
print user

In [ ]:
cust_id_unique = cust_id.unique()
print cust_id_unique.count()

In [ ]:
product_id = clean_data.map(lambda y : int(y[1]))
print product_id.take(5)

In [ ]:
product_id_unique = product_id.unique()
print product_id_unique.count()

In [ ]:
print "TRAINING AND TESTING DATA"

In [ ]:
mls = data.map(lambda l: l.split(','))

In [ ]:
ratings = mls.map(lambda x: Rating(int(x[2]), int(x[1]), float(x[3])))
print ratings.take(5)

In [ ]:
train, test = ratings.randomSplit([0.7,0.3],7856)

In [ ]:
print train.count()
print test.count()

In [ ]:
train.cache()

In [ ]:
test.cache()

In [ ]:
print "ALS MODEL"

In [ ]:
rank = 5
numIterations = 10

In [ ]:
model = ALS.train(train, rank, numIterations)

In [ ]:
model.productFeatures().first()
model.userFeatures().first()

In [ ]:
print "USERS FOR ONE PRODUCT"
print model.recommendUsers(3249,10)

In [ ]:
print "PRODUCTS FOR ONE USER"
print model.recommendProducts(17850, 10)

In [ ]:
print "PREDICTION"
print model.predict(17850,3249)

In [ ]:
print datetime.datetime.now()

In [ ]:
print "RECOMMENDATION FOR EVERY USER"
print model.recommendProductsForUsers(2).take(5)

In [ ]:
print datetime.datetime.now()

In [ ]:
pred_input = train.map(lambda x:(x[0], x[1]))
pred = model.predictAll(pred_input)
true_reorg = train.map(lambda x:((x[0],x[1]), x[2]))
pred_reorg = pred.map(lambda x:((x[0],x[1]), x[2]))
true_pred = true_reorg.join(pred_reorg)
print true_pred.take(5)

In [ ]:
print "RECOMMENDATION EVALUATION"

In [ ]:
MSE = true_pred.map(lambda r:(r[1][0] - r[1][1])**2).mean()
RMSE = sqrt(MSE)
print RMSE

In [ ]:
test_input = test.map(lambda x:(x[0],x[1]))
pred_test = model.predictAll(test_input)
test_reorg = test.map(lambda x:((x[0],x[1]), x[2]))
pred_reorg = pred_test.map(lambda x:((x[0],x[1]), x[2]))
test_pred = test_reorg.join(pred_reorg)
test_MSE = test_pred.map(lambda r: (r[1][0] - r[1][1])**2).mean()
test_RMSE = sqrt(test_MSE)
print test_RMSE